# CNN MNIST dataset

### Importing libraries and dataset

In [131]:
import pandas as pd
import numpy as np
import random
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K

In [132]:
#importing MNIST dataset from keras
from keras.datasets import mnist

In [133]:
#loading dataset into train and test sets
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [134]:
print("train data")
print(x_train.shape,y_train.shape)
print("test data")
print(x_test.shape,y_test.shape)

train data
(60000, 28, 28) (60000,)
test data
(10000, 28, 28) (10000,)


We have 60000 training images and 10000 test images each of size 28X28. Now since the images are only grey scale, the matrix representation is grey scale.

In [135]:
# selecting 20000 images from training dataset randomly to speed up training.
img=np.random.randint(x_train.shape[0],size=20000)
x_train=x_train[img,:]
y_train=y_train[img]
print(x_train.shape,y_train.shape)

(20000, 28, 28) (20000,)


## Data Preparation


### Reshaping data
Since the images are of size (28,28,1) the training data x_train needs to be of shape (20000,28,28,1). Had there been coloured images the size of the dataset would be (20000,28,28,3).

Also since all of the images have 0-9 label, y_train needs to be of shape (20000,10), where each image is represented by as a 10-d one hot encoded vector.

In [136]:
# specifying the input shape
img_rows,img_cols=28,28
input_shape=(img_rows,img_cols,1)

# batch size, number of classes, epochs
batch_size=128
num_classes=10
epochs=12

In [137]:
# reshape x_train, x_test
x_train=x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
x_test=x_test.reshape(x_test.shape[0],img_rows,img_cols,1)
print(x_train.shape,x_test.shape)

(20000, 28, 28, 1) (10000, 28, 28, 1)


In [138]:
# converting y_train and y_test to one hot encoded vectors
from keras import utils as np_utils

y_train=keras.utils.np_utils.to_categorical(y_train,num_classes)
y_test=keras.utils.np_utils.to_categorical(y_test,num_classes)
print(y_train.shape,y_test.shape)

(20000, 10) (10000, 10)


### Converting the datatypes of images from int to float
It is advisable to feed the data as float. This is not really compulsory, but advisable.

In [139]:
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')
print(x_test.dtype)
print(x_train.dtype)

float32
float32


### Normalizing
The value of each pixel is between 0-255, so we will rescale each pixel by dividing by 255 so that the range becomes 0-1. 

In [140]:
x_train /= 255
x_test /= 255

## Building Model

Building a shallow CNN having:

1) Two convolution layers with 32 and 64 filters respectively.

2) Followed by a MaxPooling layer

3) Flatten the network to give a long vector

4) Fully connected dense layer with 128 neurons

5) A softmax layer with 10 neurons

In [141]:
model=Sequential()

#keras convolution layer is called conv2D
#first convolution layer
model.add(Conv2D(32,kernel_size=(3,3),
                 activation='relu',
                 input_shape=input_shape
                ))
# second layer
model.add(Conv2D(64,kernel_size=(3,3),
                 activation='relu',
                ))
# adding maxpool layer
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Flatten and add a fully connected layer
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.5))

# softmax layer
model.add(Dense(num_classes,activation='softmax'))

# model summary
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_35 (Conv2D)          (None, 26, 26, 32)        320       
                                                                 
 conv2d_36 (Conv2D)          (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 12, 12, 64)       0         
 g2D)                                                            
                                                                 
 dropout_32 (Dropout)        (None, 12, 12, 64)        0         
                                                                 
 flatten_16 (Flatten)        (None, 9216)              0         
                                                                 
 dense_32 (Dense)            (None, 128)               1179776   
                                                     

## Fitting and evaluating Model

In [142]:
# using cross entropy loss, any optimizer like adam, rmsprop and metric is accuracy
from tensorflow import keras
model.compile(loss=keras.losses.categorical_crossentropy,
             optimizer=keras.optimizers.Adadelta(),
             metrics=['accuracy'])
# fitting the model
model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=1,
         validation_data=(x_test,y_test))

Epoch 1/12
157/157 [==============================] - 19s 120ms/step - loss: 2.3038 - accuracy: 0.1163 - val_loss: 2.2846 - val_accuracy: 0.1656
Epoch 2/12
157/157 [==============================] - 19s 118ms/step - loss: 2.2812 - accuracy: 0.1507 - val_loss: 2.2594 - val_accuracy: 0.2393
Epoch 3/12
157/157 [==============================] - 19s 119ms/step - loss: 2.2562 - accuracy: 0.1973 - val_loss: 2.2331 - val_accuracy: 0.3592
Epoch 4/12
157/157 [==============================] - 18s 116ms/step - loss: 2.2312 - accuracy: 0.2394 - val_loss: 2.2050 - val_accuracy: 0.4612
Epoch 5/12
157/157 [==============================] - 18s 117ms/step - loss: 2.2059 - accuracy: 0.2810 - val_loss: 2.1747 - val_accuracy: 0.5262
Epoch 6/12
157/157 [==============================] - 19s 122ms/step - loss: 2.1766 - accuracy: 0.3216 - val_loss: 2.1412 - val_accuracy: 0.5678
Epoch 7/12
157/157 [==============================] - 19s 119ms/step - loss: 2.1463 - accuracy: 0.3609 - val_loss: 2.1045 - val_ac

In [143]:
# evaluate the data on test data
model.evaluate(x_test,y_test)

313/313 [==============================] - 2s 7ms/step - loss: 1.8560 - accuracy: 0.7061


[1.856016755104065, 0.7060999870300293]

In [144]:
print(model.metrics_names)

['loss', 'accuracy']


The final loss (on test data) is about 1.85 and the accuracy is 70.6%